In [ ]:
%matplotlib inline
import math
import random
import matplotlib.pyplot as plt

Probabilistic data structures let us save RAM by sacrificing accuracy. Different types of probabilistic data structures do this differently, and therefore they are more useful for different applications. 

## Morris counter

A Morris counter is useful when we need to count large numbers but precission is not that important. This is how it works:

- set the counter at i = 0
- when querying the counter, return 2 ^ i, and increment i with probability 1 / 2 ^ i

So, for instance, the first time we query the counter we obtain the value 1. Next time we will get the value 2. Then we will get the value 2 or 4 with 0.5 probability each, etc. The more we count, the more inaccurate the value is. Therefore, this method is useful when we are interested to know orders of magniture.

If we only use one byte to store the value of i (so that the maximum value of i is 256), we can count up to 2 ^ 256, which would require many more bytes.

In the plot below, the continuous line represents the increment of a normal integer, which requires multiple bytes. The dotted lines represent the increment of three different 1 byte Morris counter. We compare to three runs of the Morris counter due to the random nature of the algorithm. 

In [ ]:
def morris_counter(n):
    it = 0
    i = 0
    while it < n:
        yield 2 ** i
        
        if random.random() <= 1 / (2 ** i):
            i = i + 1
            
        it = it + 1

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(10000), range(10000), lw = 2)
ax.plot(range(10000), list(morris_counter(10000)), '--')
ax.plot(range(10000), list(morris_counter(10000)), '--')
ax.plot(range(10000), list(morris_counter(10000)), '--')
fig.set_figwidth(16)